In [1]:
!pip install tensorflow scikit-learn

In [ ]:
!pip install kaggle

from google.colab import files
files.upload()  

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip gtsrb-german-traffic-sign.zip -d dataset/


Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/5/00005_00053_00010.png  
  inflating: dataset/train/5/00005_00053_00011.png  
  inflating: dataset/train/5/00005_00053_00012.png  
  inflating: dataset/train/5/00005_00053_00013.png  
  inflating: dataset/train/5/00005_00053_00014.png  
  inflating: dataset/train/5/00005_00053_00015.png  
  inflating: dataset/train/5/00005_00053_00016.png  
  inflating: dataset/train/5/00005_00053_00017.png  
  inflating: dataset/train/5/00005_00053_00018.png  
  inflating: dataset/train/5/00005_00053_00019.png  
  inflating: dataset/train/5/00005_00053_00020.png  
  inflating: dataset/train/5/00005_00053_00021.png  
  inflating: dataset/train/5/00005_00053_00022.png  
  inflating: dataset/train/5/00005_00053_00023.png  
  inflating: dataset/train/5/00005_00053_00024.png  
  inflating: dataset/train/5/00005_00053_00025.png  
  inflating: dataset/train/5/00005_00053_00026.png  
  inflating: dataset/train/5/00005_00053_00027.p

In [ ]:
!pip install tensorflow==2.19.1 pandas scikit-learn matplotlib
import os
import numpy as np
import pandas as pd
import random
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
cur_path = '/content/dataset'  
classes = 43
data = []

for i in range(classes):
    path = os.path.join(cur_path, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            img = Image.open(os.path.join(path, a))
            img = img.resize((30, 30))
            img = np.array(img)
            data.append([img, i])
        except:
            print("Error loading image:", a)

random.shuffle(data)

x = []
y = []

for features, label in data:
    x.append(features)
    y.append(label)

x = np.array(x)
y = np.array(y)

x = x / 255.0
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

print("Train images shape:", X_train.shape)
print("Validation images shape:", X_val.shape)
print("Train labels shape:", y_train.shape)
print("Validation labels shape:", y_val.shape)

aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    fill_mode="nearest"
)
model = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(30, 30, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(classes),
    Activation('softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    aug.flow(X_train, y_train, batch_size=32),
    epochs=15,
    validation_data=(X_val, y_val)
)

model.save('traffic_sign_model.h5')
class_labels = {
    0: 'Speed limit 20km/h', 1: 'Speed limit 30km/h', 2: 'Speed limit 50km/h',
    3: 'Speed limit 60km/h', 4: 'Speed limit 70km/h', 5: 'Speed limit 80km/h',
    6: 'End of speed limit 80km/h', 7: 'Speed limit 100km/h', 8: 'Speed limit 120km/h',
    9: 'No passing', 10: 'No passing veh over 3.5 tons', 11: 'Right-of-way at intersection',
    12: 'Priority road', 13: 'Yield', 14: 'Stop', 15: 'No vehicles',
    16: 'Veh > 3.5 tons prohibited', 17: 'No entry', 18: 'General caution',
    19: 'Dangerous curve left', 20: 'Dangerous curve right', 21: 'Double curve',
    22: 'Bumpy road', 23: 'Slippery road', 24: 'Road narrows on the right',
    25: 'Road work', 26: 'Traffic signals', 27: 'Pedestrians', 28: 'Children crossing',
    29: 'Bicycles crossing', 30: 'Beware of ice/snow', 31: 'Wild animals crossing',
    32: 'End speed + passing limits', 33: 'Turn right ahead', 34: 'Turn left ahead',
    35: 'Ahead only', 36: 'Go straight or right', 37: 'Go straight or left',
    38: 'Keep right', 39: 'Keep left', 40: 'Roundabout mandatory',
    41: 'End of no passing', 42: 'End no passing veh > 3.5 tons'
}

y_test_csv = pd.read_csv(os.path.join(cur_path, 'Test.csv')) 
labels_test = y_test_csv['ClassId'].values
imgs_test = y_test_csv['Path'].values

X_test = []
for img_path in imgs_test:
    full_path = os.path.join(cur_path, img_path)
    img = Image.open(full_path)
    img = img.resize((30, 30))
    X_test.append(np.array(img))

X_test = np.array(X_test)/255.0

y_pred = np.argmax(model.predict(X_test), axis=-1)

acc = accuracy_score(labels_test, y_pred)
f1 = f1_score(labels_test, y_pred, average='weighted')

print("Test Accuracy:", acc)
print("Weighted F1-score:", f1)
from tensorflow.keras.preprocessing import image

def predict_single(img_path):
    img = Image.open(img_path).resize((30,30))
    x = np.expand_dims(np.array(img)/255.0, axis=0)
    pred_class = np.argmax(model.predict(x), axis=-1)
    print("Predicted class:", class_labels[pred_class[0]])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 915.9 kB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.19.1 which is incompatible.
Train images shape: (31367, 30, 30, 3)
Validation images shape: (7842, 30, 30, 3)
Train labels shape: (31367,)
Validation labels shape: (7842,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


981/981 ━━━━━━━━━━━━━━━━━━━━ 83s 83ms/step - accuracy: 0.1844 - loss: 3.0158 - val_accuracy: 0.6038 - val_loss: 1.2400
Epoch 2/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 79s 80ms/step - accuracy: 0.5568 - loss: 1.3338 - val_accuracy: 0.8347 - val_loss: 0.5279
Epoch 3/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 78s 80ms/step - accuracy: 0.7363 - loss: 0.7809 - val_accuracy: 0.9236 - val_loss: 0.2490
Epoch 4/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 80s 82ms/step - accuracy: 0.8308 - loss: 0.5033 - val_accuracy: 0.9523 - val_loss: 0.1515
Epoch 5/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 78s 79ms/step - accuracy: 0.8774 - loss: 0.3757 - val_accuracy: 0.9713 - val_loss: 0.1075
Epoch 6/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 86s 88ms/step - accuracy: 0.9086 - loss: 0.2781 - val_accuracy: 0.9832 - val_loss: 0.0662
Epoch 7/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 77s 79ms/step - accuracy: 0.9227 - loss: 0.2410 - val_accuracy: 0.9815 - val_loss: 0.0581
Epoch 8/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 77s 79ms/step - accuracy: 0.9349 - loss: 0.2018 - val_accurac

395/395 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step
Test Accuracy: 0.9510688836104513
Weighted F1-score: 0.9509287529296906
